# define labels and extract activity patterns binned at different durations. Also extract patterns of cells registered across sessions

In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

In [2]:
# all modules necessary for this nb
import os
import sys
import pickle

import numpy as np
import pylab as pl

# setting parameters for default matplotlib plots
%matplotlib inline

In [3]:
pl.style.use('fabiostyle')
# needs to find the library of functions
sys.path.append('../../../../code/')  # to be replaced!

import utils as ut
import plots as pt

In [4]:
NOTEBOOK_NAME = 'patterns_time_bins'

In [5]:
from pickleshare import PickleShareDB

autorestore_folder = os.path.join(os.getcwd(), 'autorestore', NOTEBOOK_NAME)
db = PickleShareDB(autorestore_folder)
import sys
from workspace import *
import IPython
ip = IPython.get_ipython()

# this will restore all the saved variables. ignore the errors listed.
load_workspace(ip, db)

# use `save_worspace(db)` to save variables at the end

In [6]:
sessions = 'expo','pre','post','post_hiatus'

In [7]:
#import variables.
for sess in sessions:
    #try:
        if sess == 'expo':
            folder = '../4_odor_expo/notebooks/autorestore/preprocessing'
        else:
            exec("folder = '../%s/notebooks/autorestore/preprocessing'")%sess
        for var in ('time_ax', 'traces', 'events', 'cycles',
                'is_STIM1t', 'is_STIM2t', 'is_STIM3t','is_STIM4t','rewards','licks',
                'CYCLE_START','CS_DURATION','DELAY','US_START'): 
            exec('%s_%s = ut.load_variable(\'%s\', folder=\'%s\')'%(var, sess, var, folder))
    #except:
    #    print ('%s failed')%sess

In [8]:
#assign labels
for sess in sessions:
    #try:
        exec('labels_%s = ((np.r_[is_STIM1t_%s]*1)+(np.r_[is_STIM2t_%s]*2)+(np.r_[is_STIM3t_%s]*3)+(np.r_[is_STIM4t_%s]*4))'%(sess,sess,sess,sess,sess))
    #except:
    #    print'nope %s'%(sess)

In [9]:
#0.5sec time bins
delta_t0 = 0.5
t_start0 = -5.5  # sec
t_stop0 = 10.5  # sec

for sess in sessions:
    #try:
        exec("patterns_evs_05s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    #except:
    #    print 'nope %s'%(sess)

In [10]:
#1sec time bins
delta_t0 = 1
t_start0 = -5  # sec
t_stop0 = 11  # sec

for sess in sessions:
    #try:
        exec("patterns_evs_1s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    #except:
    #    print 'nope %s'%(sess)

In [11]:
#1sec time bins centered on the half-second
delta_t0 = 1
t_start0 = -5.5  # sec
t_stop0 = 10.5  # sec

for sess in sessions:
    #try:
        exec("patterns_evs_15s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    #except:
    #    print 'nope %s'%(sess)

In [12]:
#2sec time bins
delta_t0 = 2
t_start0 = -4  # sec
t_stop0 = 10  # sec

for sess in sessions:
    #try:
        exec("patterns_evs_2s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    #except:
    #    print 'nope %s'%(sess)

In [13]:
#replace all nan values with 0
for tb in ['05s','1s','15s','2s']:
    for sess in sessions:
        try:
            for x in np.argwhere(np.isnan(eval("patterns_evs_%s_%s"%(tb,sess)))):
                print x,
                exec("patterns_evs_%s_%s[x[0],x[1],x[2]]=0"%(tb,sess))
        except:
            print 'nope',tb,sess

In [14]:
#make sure no more nan values remain (nothing should be printed when run)
for tb in ['05s','1s','15s','2s']:
        for sess in sessions:
            try:
                for x in np.argwhere(np.isnan(eval("patterns_evs_%s_%s"%(tb,sess)))):
                    print x,
            except:
                print 'nope',tb,sess

CR_comparisons = 'pre_vs_post','pre_vs_post_hiatus','post_vs_post_hiatus','all' #exclude expo session, since did not cellreg this

In [15]:
CR_comparisons = [['pre','post'],['pre','post_hiatus'],['post','post_hiatus']] #exclude expo session, since did not cellreg this

In [16]:
# import cells registered across session pairs
for cr in CR_comparisons:
    try:
        exec("cellregs_%s_vs_%s = ut.load_variable('cellregs_%s_vs_%s', folder='../notebooks/autorestore/cellregs')"%(cr[0],cr[1],
                                                                                                                cr[0],cr[1]))
    except:
        print ('%s_vs_%s failed')%cr[0],cr[1]

In [17]:
# import cells registered across all 3 sessions.
cellregs_all = ut.load_variable('cellregs_all', folder='../notebooks/autorestore/cellregs')

In [18]:
#extract patterns using only registered cells
for tb in ['05s','1s','15s','2s']:
    for cr in CR_comparisons:
            #try:
                exec("patterns_evs_reg_%s_%s_%s = patterns_evs_%s_%s[:, :, cellregs_%s_vs_%s['%s']]"%(tb,cr[0],cr[1],
                                                                  tb,cr[0],cr[0],cr[1],cr[0]))
                exec("patterns_evs_reg_%s_%s_%s = patterns_evs_%s_%s[:, :, cellregs_%s_vs_%s['%s']]"%(tb,cr[1],cr[0],
                                                                  tb,cr[1],cr[0],cr[1],cr[1]))
            #except:
            #    print 'nope %s'%tb,cr,

In [19]:
#extract patterns using only cells registered across all 3 sessions (pre, post, post_hiatus)
for tb in ['05s','1s','15s','2s']:
    for sess in sessions[1:]: #exclude expo session (cuz didn't register those)
            #try:
                exec("patterns_evs_reg_%s_%s_all = patterns_evs_%s_%s[:, :, cellregs_all['%s']]"%(tb,sess,tb,sess,sess))
                exec("patterns_evs_reg_%s_%s_all = patterns_evs_%s_%s[:, :, cellregs_all['%s']]"%(tb,sess,tb,sess,sess))
            #except:
            #    print 'nope %s'%tb,cr,

In [20]:
save_workspace(db);

Could not store variable 'di'. Skipping...
Could not store variable 'os'. Skipping...
Could not store variable 'IPython'. Skipping...
Could not store variable 'pt'. Skipping...
Could not store variable 'pl'. Skipping...
Could not store variable 'ut'. Skipping...
Could not store variable 'pickle'. Skipping...
Could not store variable 'ip'. Skipping...
Could not store variable 'np'. Skipping...
Could not store variable 'sys'. Skipping...
